In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/2b/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b/2/gemma-2b.gguf
/kaggle/input/gemma/transformers/2b/2/config.json
/kaggle/input/gemma/transformers/2b/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/tokenizer.json
/kaggle/input/gemma/transformers/2b/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b/2/.gitattributes
/kaggle/input/gemma/transformers/2b/2/tokenizer.model
/kaggle/input/gemma/transformers/2b/2/generation_config.json


In [2]:
import pandas as pd

df = pd.read_json("hf://datasets/giuliadc/cnndm_5k/cnndm_5k_2-0.630-test.json")

In [51]:
def make_model():
    try:
        del model
        for _ in range(10):
            torch.cuda.empty_cache()
            gc.collect()
    except:
        pass
    model = GemmaForCausalLM.from_pretrained(model_name)  # Use the appropriate model class
    model.to(device)
    return model

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, GemmaForCausalLM 
import torch
model_name = '/kaggle/input/gemma/transformers/2b/2'  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = make_model()

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
df.head()

,id,text,reference-summary,source
0,cnndm-1,A flickering rainbow of colours came with the ...,"Italian photographer Giovanna Griffo, 42, shar...",4ab4bb5edcb0a9b97b87a0c4a9a721984210c2f4
1,cnndm-2,Ed Sheeran surprised a fan in hospital for her...,"Jess Knight, 20, was surprised by Ed Sheeran a...",ac1b0180881c97d9f0d57d403c0bbc21fe29dd3c
2,cnndm-3,It’s been one of the most indulgent shopping s...,"Lily James, the British star of the upcoming C...",8f3a478ae49dfe072a204b60c2577c2375341628
3,cnndm-4,The Block’s up market South Yarra apartments a...,The Block’s four apartments are now on the mar...,3f87b31923a09bf2248f7b3bc3303bb0c9ebdbe4
4,cnndm-5,"(CNN)Recently, a New York judge issued an opin...",A court allowed a wife to serve divorce papers...,8610ead0a43054be202a1fc756620415a6d572fe


In [6]:
!pip install bert-score
!pip install rouge-score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import nltk
from transformers.modeling_utils import prune_linear_layer
from collections import defaultdict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d61f6488b8ebcaab3711cca1f543c587b1c1239ceea0f2074d30a2828edf6125
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [7]:
# Download required packages for BLEU
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
def summarize_text(text, model):
    inputs = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True).to("cuda")
    
    # Use max_new_tokens to control the output length while keeping input handling flexible
    summary_ids = model.generate(
        inputs.input_ids, 
        max_new_tokens=150,  # Generates up to 150 tokens in the summary
        min_length=50, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True
    )
    res = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    del inputs
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [9]:
# from transformers import AutoTokenizer, GemmaForCausalLM
# Take the first 100 rows

df_100 = df.head(30)
# df_100.to(device)
# Apply the summarization to the 'text' column for the first 100 rows
df_100['generated_summary'] = df_100['text'].apply(summarize_text, model=model)


/tmp/ipykernel_30/393478883.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['generated_summary'] = df_100['text'].apply(summarize_text)


In [10]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to compute BLEU and ROUGE scores
def compute_scores(reference, hypothesis):
    # BLEU score
    bleu_score = sentence_bleu([reference.split()], hypothesis.split())
    
    # ROUGE score
    rouge_scores = scorer.score(reference, hypothesis)
    
    return bleu_score, rouge_scores

# List to store scores
results = []

# Iterate through the first 100 rows to compute BLEU and ROUGE scores
for i, row in df_100.iterrows():
    reference = row['reference-summary']
    hypothesis = row['generated_summary']
    
    # Calculate BLEU and ROUGE scores
    bleu_score, rouge_scores = compute_scores(reference, hypothesis)
    
    # Store results
    results.append({
        'id': row['id'],
        'bleu_score': bleu_score,
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rouge2': rouge_scores['rouge2'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display results
print(results_df.head())


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


        id  bleu_score    rouge1    rouge2    rougeL
0  cnndm-1    0.012572  0.093126  0.044444  0.082040
1  cnndm-2    0.024412  0.154278  0.067511  0.109397
2  cnndm-3    0.012623  0.113350  0.045455  0.080605
3  cnndm-4    0.019106  0.147087  0.065072  0.093601
4  cnndm-5    0.067139  0.032020  0.007407  0.022167


In [11]:
from bert_score import score

# Assuming df_100 has 'generated_summary' and 'reference-summary' columns
generated_summaries = df_100['generated_summary'].tolist()
reference_summaries = df_100['reference-summary'].tolist()

# Calculate BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang='en', verbose=True)

# Output the scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.14 seconds, 9.56 sentences/sec
Precision: 0.8032
Recall: 0.8843
F1 Score: 0.8418


In [12]:
import random
def create_random_binary_list(length, percentage_of_zeros):
    num_zeros = int(length * percentage_of_zeros)
    num_ones = length - num_zeros

    # Create the list with the required number of 0s and 1s
    binary_list = [0] * num_zeros + [1] * num_ones

    # Shuffle the list to randomize the order
    random.shuffle(binary_list)

    return binary_list

def initialize_chromosome(num_genes):
  # initialize chromosome with given sparsity percentage
  return create_random_binary_list(num_genes, SPARSITY_RATE)

In [13]:
def modify_model(model, chromosome):
    num_heads = model.config.num_attention_heads
    num_blocks = model.config.num_hidden_layers
    # num_ffn = model.config.num_hidden_layers

    # Disable attention heads
    heads_to_prune = defaultdict(list)
    for i, gene in enumerate(chromosome):
        if gene == 0:
          block_num = i//num_heads
          head_num = i%num_heads
          heads_to_prune[block_num].append(head_num)

    head_dim = model.config.head_dim
    if heads_to_prune:
        print("Pruning heads in model")
        with torch.no_grad():
            for block in range(block_num): 
                for head in heads_to_prune[block]:
                        # Zero-out the corresponding rows in the q_proj, k_proj, and v_proj
                        start_index = head * head_dim
                        end_index = (head + 1) * head_dim
                        model.model.layers[block].self_attn.q_proj.weight[start_index:end_index, :] = 0
                        model.model.layers[block].self_attn.k_proj.weight[start_index:end_index, :] = 0
                        model.model.layers[block].self_attn.v_proj.weight[start_index:end_index, :] = 0

    #                     model.model.layers[block_num].self_attn.q_proj.weight = model.model.layers[block_num].self_attn.q_proj.weight.to(torch.int8)
    #                     model.model.layers[block_num].self_attn.k_proj.weight = model.model.layers[block_num].self_attn.q_proj.weight.to(torch.int8)
    #                     model.model.layers[block_num].self_attn.c_proj.weight = model.model.layers[block_num].self_attn.q_proj.weight.to(torch.int8)


    return model

In [45]:
def evaluate_fitness(chromosome, dataset):
    model = make_model()
    model = modify_model(model,chromosome)
    df_100['generated_summary'] = dataset['text'].apply(summarize_text, model=model)
    df_100['reference-summary'] = dataset['reference-summary']
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # List to store scores
    results = []
    
    # Iterate through the first 100 rows to compute BLEU and ROUGE scores
    for i, row in df_100.iterrows():
        reference = row['reference-summary']
        hypothesis = row['generated_summary']
        
        # Calculate BLEU and ROUGE scores
        bleu_score, rouge_scores = compute_scores(reference, hypothesis)
        
        # Store results
        results.append({
            'id': row['id'],
            'bleu_score': bleu_score,
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rouge2': rouge_scores['rouge2'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure
        })
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    generated_summaries = df_100['generated_summary'].tolist()
    reference_summaries = df_100['reference-summary'].tolist()

    # Calculate BERTScore
    P, R, F1 = score(generated_summaries, reference_summaries, lang='en', verbose=True)
    return P.mean().item()


def compute_scores(reference, hypothesis):
    # BLEU score
    bleu_score = sentence_bleu([reference.split()], hypothesis.split())
    
    # ROUGE score
    rouge_scores = scorer.score(reference, hypothesis)
    
    return bleu_score, rouge_scores

In [15]:
SPARSITY_RATE = 0.3

In [16]:
def find_size(model):
  total_size_in_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
  total_size_in_megabytes = total_size_in_bytes / (1024 ** 2)
  print(f"Model size: {total_size_in_megabytes:.2f} MB")

In [26]:
model = make_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
chrom = initialize_chromosome(model.config.num_attention_heads*model.config.num_hidden_layers)
# model = modify_model(model, chrom)
# find_size(model)

In [30]:
import gc
del model
del df
for _ in range(10):
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
!nvidia-smi


In [ ]:
# model = make_model()
# find_size(model)

In [36]:
evaluate_fitness(chrom, df_100)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Pruning heads in model


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.06 seconds, 9.80 sentences/sec


0.8033027648925781

In [37]:
POPN_SIZE = 8
crossover_rate = 0.7
mutation_rate = 0.08


In [38]:
def initialize_population(chromosome_length):
  # initialize random population
  population = []
  for _ in range(POPN_SIZE):
    chromosome = initialize_chromosome(chromosome_length)
    population.append(chromosome)
  return population

In [39]:
def select_parents(population, fitness_scores, num_parents):
    # Normalize fitness scores to create a probability distribution
    total_fitness = np.sum(fitness_scores)
    probabilities = fitness_scores / total_fitness

    # Select parents based on their fitness proportion (roulette wheel selection)
    # selected_parents = np.random.choice(population, size=num_parents, p=probabilities, replace=True)
    selected_parents = random.choices(population, weights=probabilities, k=num_parents)


    return np.array(selected_parents)

In [40]:
# Crossover (Single-point crossover)
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate([parent1[:point], parent2[point:]])
        child2 = np.concatenate([parent2[:point], parent1[point:]])
    else:
        child1, child2 = parent1, parent2
    return child1, child2

In [41]:
# Mutation (Flip bit mutation)
def mutate(chromosome):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]

    target_zeros = int(len(chromosome) * SPARSITY_RATE)

    for c in range(model.config.num_attention_heads-1, len(chromosome), model.config.num_attention_heads):
      # this part to ensure that each layer has at least one attention head
      start = c-model.config.num_attention_heads-1
      enc_part = chromosome[start:c]
      num_ones = np.sum(enc_part)  # Count the number of 1s in the chromosome
      if num_ones==0:
        chromosome[start] = 1

    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            if chromosome[i] == 1 and num_ones > target_ones:
                chromosome[i] = 0  # Flip 1 to 0 only if there are too many 1s
                num_ones -= 1
            elif chromosome[i] == 0 and num_ones < target_ones:
                chromosome[i] = 1  # Flip 0 to 1 only if there are too few 1s
                num_ones += 1
    return chromosome


In [42]:
def elitism_and_selection(population, fitness_scores, num_elites, num_parents):
    # Elitism: Keep the top num_elites individuals
    elite_indices = np.argsort(fitness_scores)[-num_elites:]  # Get indices of top individuals
    elites = [population[i] for i in elite_indices]

    # Perform roulette wheel selection for the rest of the parents
    remaining_population = np.delete(population, elite_indices, axis=0)
    remaining_fitness_scores = np.delete(fitness_scores, elite_indices)

    num_to_select = num_parents - num_elites
    selected_parents = select_parents(remaining_population, remaining_fitness_scores, num_to_select)

    # Combine elites and selected parents
    next_generation = np.vstack((elites, selected_parents))

    return next_generation

In [56]:
model = make_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
num_attention_heads = model.config.num_attention_heads
num_hidden_layers = model.config.num_hidden_layers

In [59]:
import matplotlib.pyplot as plt

def genetic_algorithm(num_generations, desired_sparsity):
  population = initialize_population(num_attention_heads*num_hidden_layers)  # Initialize the population
  accuracy_per_generation = []  # List to store highest accuracy values for each generation
  for generation in range(num_generations):
      fitness_scores = np.array([evaluate_fitness(chrom, df_100) for chrom in population])
      best_chromosome = population[np.argmax(fitness_scores)]
      print("new fitness scores:", fitness_scores)
      print(f"best chromosome in generation {generation} is {best_chromosome} with accuracy {fitness_scores[np.argmax(fitness_scores)]}")
      accuracy_per_generation.append(fitness_scores[np.argmax(fitness_scores)])
      parents = elitism_and_selection(population, fitness_scores, 4, POPN_SIZE)
      # parents = select_parents(population, fitness_scores, POPN_SIZE)
      new_population = []
      for i in range(0, POPN_SIZE, 2):
          parent1, parent2 = parents[i], parents[i + 1]
          child1, child2 = crossover(parent1, parent2)
          # child1 = mutate(child1)
          # child2 = mutate(child2)
          new_population.extend([child1, child2])
      population = np.array(new_population)
  generations = list(range(1, len(accuracy_per_generation) + 1))


  return best_chromosome

In [66]:
genetic_algorithm(2, SPARSITY_RATE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Pruning heads in model


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.96 seconds, 10.14 sentences/sec
new fitness scores: [0.8026759  0.80302864 0.80277288 0.80306411 0.80203593 0.80127829
 0.80115384 0.80115384 0.80230469 0.8010906  0.80110371 0.80110371
 0.80250347 0.80101788 0.80230826 0.8010596 ]
best chromosome in generation 1 is [1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 0
 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1
 0 1 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1] with accuracy 0.8030641078948975


array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])